# 2023C3: Plotting SST1 RSoXS Data

## Imports

In [ ]:
!pip install -e /nsls2/users/alevin/repos/PyHyperScattering  # to use pip to install via directory

In [ ]:
## Imports
import PyHyperScattering as phs
import pathlib
import sys
import io
import ast
import json
import datetime
import dask.array as da
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from tqdm.auto import tqdm
import dask.array as da
from tiled.client import from_profile, from_uri
import subprocess


print(f'Using PyHyperScattering Version: {phs.__version__}')

## Define Paths

In [ ]:
def make_para_perp_DAs(datasets, sample_name, edge, intensity_type, pol, qlims, chi_width):
    # select dataarray to plot
    DS = datasets[f'polar_{edge}']
    DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
    sliced_DA = DA.sel(polarization=pol, q=slice(qlims[0],qlims[1]))

    # calculate ISI dataarrays
    if pol==0:
        para_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        perp_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))
    elif pol==90:
        perp_DA = sliced_DA.rsoxs.slice_chi(180, chi_width=(chi_width/2))
        para_DA = sliced_DA.rsoxs.slice_chi(90, chi_width=(chi_width/2))   
        
    return para_DA, perp_DA

In [ ]:
## Define directory paths
userPath = pathlib.Path('/nsls2/users/alevin')
propPath = pathlib.Path('/nsls2/data/sst/proposals/2023-3/pass-313412')

outPath = propPath.joinpath('processed_data')
plotsPath = outPath.joinpath('prsoxs_plots')
# jsonPath = outPath.joinpath('local_config')
maskPath = outPath.joinpath('masks')
zarrsPath = outPath.joinpath('zarrs')


# propPath = pathlib.Path('/nsls2/data/sst/proposals/2022-2/pass-309180')
# zarrsPath = propPath.joinpath('zarr_datasets_jul2022')
# outPath = propPath.joinpath('processed_data')
# plotsPath = outPath.joinpath('prsoxs_plots')

In [ ]:
plotsPath

## Load DataArrays 

In [ ]:
# load dictionary of rsoxs datasets
rsoxs_datasets = {}
for key in ['cartesian_carbon', 'polar_carbon']:
    key_start = key.split('_')[0]
    key_end = key.split('_')[1]        
    zarrPath = list(zarrsPath.glob(f'{key_start}*{key_end}_rechunked-v2.zarr'))[0]
    rsoxs_datasets[key] = xr.open_zarr(zarrPath).sortby('sample_name')
    # rsoxs_datasets[key] = rsoxs_datasets[key].drop('dark_id')
    
    # Compute any dask coordiantes
    for coord_name, coord_data in rsoxs_datasets[key].coords.items():
        if isinstance(coord_data.data, da.Array):
            rsoxs_datasets[key].coords[coord_name] = coord_data.compute()

In [ ]:
DS = rsoxs_datasets['polar_carbon']
DS

In [ ]:
DS

In [ ]:
DS.sel(pix_x=slice(500,600), pix_y=slice(500,650))

In [ ]:
# sorted([f.name for f in zarrsPath.glob('TRMSN*nexafs*')])

In [ ]:
trmsn_4angle = xr.open_zarr(zarrsPath.joinpath('TRMSN_nexafs_carbon_day1.zarr'))
trmsn_3angle = xr.open_zarr(zarrsPath.joinpath('TRMSN_nexafs_carbon_3angle.zarr'))

trmsn_90_part1 = trmsn_4angle.sel(theta=90)
trmsn_90_part2 = trmsn_3angle.sel(theta=90)

trmsn90 = xr.concat([trmsn_90_part1, trmsn_90_part2], dim='sample_name')
trmsn90

In [ ]:
# polar_netcdf_paths = str(zarrsPath.joinpath('polar_rsoxs_carbon_ncs')) + '/*.nc'
# polar_netcdf_paths

In [ ]:
# # rsoxs_datasets = {}
# # rsoxs_datasets['polar_carbon'] = xr.open_mfdataset(polar_netcdf_paths, parallel=True)
# rsoxs_datasets['polar_carbon'] = xr.open_mfdataset(polar_netcdf_paths)
# rsoxs_datasets['polar_carbon']

In [ ]:
trmsn90_sample_names = []
for sample_name in trmsn90.sample_name.values:
    trmsn90_sample_names.append(sample_name.replace('_rot', ''))
    
display(trmsn90_sample_names)

In [ ]:
bad_trmsn_samples = ['Y6_CBCN', 'Y6BO_CBCN', 'Y6BO_CFCN','Y7_CB_2000', 'Y7_CB_2000_1', 'Y7_CBCN', 'Y7_CFCN', 
                     'Y7BO_CFCN', 'BareSiN_1p5mm', 'BareSiN_1mm', 'BareSiN_1mm_1', 'BareSiN_1mm_2']

In [ ]:
selected_samples = [sample_name for sample_name in rsoxs_datasets['polar_carbon'].sample_name.values if sample_name not in bad_trmsn_samples]
selected_samples

In [ ]:
# for sample_name in selected_samples:
#     print(sample_name)
#     sample_name = sample_name.replace('_1', '')
#     print(sample_name in trmsn90_sample_names)
#     print('\n')

In [ ]:
trmsn90_DA = trmsn90['full_corr_trmsn']
# pre_edge_mean = trmsn90_DA.sel(energy=slice(270,280)).mean('energy')
# trmsn90_DA = (trmsn90_DA - pre_edge_mean) * -1

trmsn90_DA

In [ ]:
for sample_name in trmsn90_DA.sample_name.values:
    trmsn90_DA.sel(sample_name=sample_name).plot()
    plt.show()

In [ ]:
polar_DS = rsoxs_datasets['polar_carbon']
trmsn90_DA = trmsn90_DA.interp({'energy': polar_DS.energy.values})
# polar_DS

In [ ]:
fl_bkgs = []
trmsn90_bkgs = []

for sample_name in tqdm(selected_samples):
    polar_samp_DA = polar_DS['corr_intensity'].sel(sample_name=sample_name)
    fl_bkg_DA = 1 * polar_samp_DA.sel(polarization=0,q=slice(0.02, 0.06)).mean('chi').integrate('q')
    fl_bkgs.append(fl_bkg_DA)
    # fl_bkg_DA.plot()
    # plt.show()
    
    trmsn90_samp_DA = trmsn90_DA.sel(sample_name=(sample_name.replace('_1','')+'_rot')).drop_vars(['sample_id', 'sample_name', 'theta', 'scan_id'])
    trmsn90_samp_DA = trmsn90_samp_DA.expand_dims({'sample_name':[sample_name]})
    trmsn90_bkgs.append(trmsn90_samp_DA)
#     trmsn90_samp_DA.plot()
#     plt.show()
    
#     plt.close('all')
        

In [ ]:
fl_bkgs

In [ ]:
# DS['alo_sub_raw_intensity'] = DS['raw_intensity'] - DS['raw_intensity'].sel(sample_name='BareAlO')
# DS['alo_sub_corr_intensity'] = DS['corr_intensity'] - DS['corr_intensity'].sel(sample_name='BareAlO')

cmap = plt.cm.turbo.copy()
cmap.set_bad(cmap.get_under())

### Load solution SAXS PM6 data text files

In [ ]:
# Check files
saxsPath = pathlib.Path('/nsls2/data/sst/proposals/2023-3/pass-313412/processed_data/prsoxs_plots/carbon/PM6_solution_saxs_txts')
# saxsPath = outPath.joinpath('prsoxs_plots/carbon/PM6_solution_saxs_txts')
[f.name for f in sorted(saxsPath.glob('*'))]

In [ ]:
# Ensure q values are equal between datasets, else interpolation needed
print(np.array_equal(pm6_cb[:,0], pm6_cf[:,0]))
print(np.array_equal(pm6_cf[:,0], pm6_cn[:,0]))

In [ ]:
# Load data into numpy arrays
pm6_cb, pm6_cf, pm6_cn = [np.loadtxt(f) for f in sorted(saxsPath.glob('*'))]

# Put numpy arrays into DataArrays
cb_DA = xr.DataArray(data=pm6_cb[:,1], dims=['q'], coords={'q':pm6_cb[:,0], 'err':('q', pm6_cb[:,2])})
cf_DA = xr.DataArray(data=pm6_cf[:,1], dims=['q'], coords={'q':pm6_cf[:,0], 'err':('q', pm6_cf[:,2])})
cn_DA = xr.DataArray(data=pm6_cn[:,1], dims=['q'], coords={'q':pm6_cn[:,0], 'err':('q', pm6_cn[:,2])})

# Move DataArrays into Dataset
pm6_saxs_DS = xr.Dataset(attrs={'name':'PM6_Solution_SAXS'})
pm6_saxs_DS['CB'] = cb_DA
pm6_saxs_DS['CF'] = cf_DA
pm6_saxs_DS['CN'] = cn_DA
pm6_saxs_DS

In [ ]:
# qmin = float(full_DS.q.min())
qmin = 0.008
qmax = float(full_DS.q.max())
# qmax = None

plt.close('all')
fig, ax = plt.subplots()
for solvent in ['CB', 'CF', 'CN']:
    pm6_saxs_DS[solvent].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=solvent)

ax.set_ybound(lower=7e-5)
ax.legend()
plt.show()

## Intensity Plots

### 1. Detector movies

In [ ]:
# cartesian_netcdf_paths = str(zarrsPath.joinpath('cartesian_rsoxs_carbon_ncs')) + '/*.nc'
# cartesian_netcdf_paths

In [ ]:
# rsoxs_datasets = {}
# # rsoxs_datasets['polar_carbon'] = xr.open_mfdataset(polar_netcdf_paths, parallel=True)
# rsoxs_datasets['cartesian_carbon'] = xr.open_mfdataset(cartesian_netcdf_paths)
# rsoxs_datasets['cartesian_carbon']

In [ ]:
# rsoxs_datasets['cartesian_carbon']['corr_intensity'] = ((rsoxs_datasets['cartesian_carbon']['raw_intensity'] / rsoxs_datasets['cartesian_carbon']['sample_au_mesh'])
#                                                         * (rsoxs_datasets['cartesian_carbon']['calib_au_mesh'] / rsoxs_datasets['cartesian_carbon']['calib_diode']))

In [ ]:
# Select Dataset
edge = 'carbon'
DS = rsoxs_datasets[f'cartesian_{edge}'].copy()
bcx = DS['raw_intensity'].beamcenter_x
bcy = DS['raw_intensity'].beamcenter_y


# Select Plotting Parameters
# pol = 90
energy = 285
# energy=400
pix_size = 500
pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
cmin = 1e9
cmax = 1e11
# cmin = 4e1
# cmax = 5e4

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
for pol in [0]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Pol = {pol}°'):
        intensity_type = 'corr'
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']
        # cmin = float(DA.where(sliced_DA>0).compute().quantile(0.35))
        # cmax = float(DA.where(sliced_DA>0).compute().quantile(0.9999))

        # Plot
        sliced_DA = DA.sel(polarization=pol, pix_x=pix_x_slice, pix_y=pix_y_slice).sel(energy=energy,method='nearest')
        # print(f'cmin={cmin}, cmax={cmax}')
        ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='qx', y='qy', cmap=cmap, norm=LogNorm(cmin,cmax))
        ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
        ax.figure.set_tight_layout(True)
        ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
        ax.colorbar.set_label('Double-Norm-Corrected Intensity [arb. units]', rotation=270, labelpad=12)
        # ax.figure.savefig(plotsPath.joinpath('detector_movies_carbon_v1', f'{sample_name}_{edge}_{intensity_type}_pol{pol}deg.jpeg'), dpi=120)
        plt.show()
        plt.close('all')

In [ ]:
plotsPath

In [ ]:
filtered_selected_samples

In [ ]:
# Select Dataset
edge = 'carbon'
DS = rsoxs_datasets[f'cartesian_{edge}']
bcx = DS['raw_intensity'].beamcenter_x
bcy = DS['raw_intensity'].beamcenter_y

# Select Plotting Parameters
# pol = 90
pix_size = 500
pix_x_slice = slice(bcx-(pix_size/2), bcx+(pix_size/2))
pix_y_slice = slice(bcy-(pix_size/2), bcy+(pix_size/2))
intensity_type = 'corr'

# Select DataArray
# sample_name = 'PM6-Y6_3000_dSiN'
for pol in [0, 90]:
    for sample_name in tqdm(DS.sample_name.values, desc=f'Polarization {pol}°'):
        DA = DS.sel(sample_name=sample_name)[f'{intensity_type}_intensity']

        output_path = plotsPath.joinpath('detector_movies_carbon_v1', f'{sample_name}_{edge}_{intensity_type}_pol{pol}deg.mp4')

        # FFmpeg command. This is set up to accept data from the pipe and use it as input, with PNG format.
        # It will then output an H.264 encoded MP4 video.
        cmd = [
            'ffmpeg',
            '-y',  # Overwrite output file if it exists
            '-f', 'image2pipe',
            '-vcodec', 'png',
            '-r', '23',  # Frame rate
            '-i', '-',  # The input comes from a pipe
            '-vcodec', 'libx264',
            '-pix_fmt', 'yuv420p',
            '-crf', '22',  # Set the quality (lower is better, 17 is often considered visually lossless)
            str(output_path)
        ]

        # Start the subprocess
        proc = subprocess.Popen(cmd, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Loop through the energy dimension and send frames to FFmpeg
        for energy in tqdm(DA.energy.values, desc=f'Making the {sample_name} movie'):
            # Make & customize plot
            sliced_DA = DA.sel(polarization=pol, pix_x=pix_x_slice, pix_y=pix_y_slice).sel(energy=energy, method='nearest')
            ax = sliced_DA.plot.imshow(figsize=(5.5,4.5), x='qx', y='qy', cmap=cmap, norm=LogNorm(cmin,cmax))
            ax.figure.suptitle(f'Photon Energy = {np.round(energy, 1)} eV', fontsize=14, y=0.96)
            ax.figure.set_tight_layout(True)   
            ax.axes.set(aspect='equal', title=f'{sample_name}, Polarization = {pol}°', xlabel='q$_x$ [$Å^{-1}$]', ylabel='q$_y$ [$Å^{-1}$]')
            ax.colorbar.set_label('Double-Norm-Corrected Intensity [arb. units]', rotation=270, labelpad=12)

            # Save figure to a PNG buffer
            # ax.figure.savefig(plotsPath.joinpath('detector_movies/frames', f'energy-{round(energy)}_frame-{i:03d}.png'), dpi=120)

            buf = io.BytesIO()
            ax.figure.savefig(buf, format='png')
            buf.seek(0)

            # Write the PNG buffer data to the process
            proc.stdin.write(buf.getvalue())
            plt.close('all')

        # Finish the subprocess
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(f"Error: {err}")


### 2. ISI Plots

In [ ]:
# DA = rsoxs_datasets['polar_carbon']['corr_intensity']
# DA.sel(sample_name='Y6_CB_3000', polarization=0, q=slice(0.001, 0.03)).sel(energy=398.8, method='nearest').plot(norm=LogNorm(1e9, 1e11), cmap=plt.cm.turbo)
# plt.show()

In [ ]:
outPath

In [ ]:
# make selection
# sample_name = 'PM6-Y6_3000_dSiN'
edge = 'carbon'
intensity_type = 'corr'
pol = 0
qlims = (7.5e-3, None)
qslice = (1e-2, 8e-2)
chi_width = 90
figsize = (6,4)

energy_min = 282
energy_max = 290

for sample_name in tqdm(filtered_selected_samples):
# for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'][f'{intensity_type}_intensity'].sortby('sample_name').sample_name.values):
# for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.values[:]):
    # for pol in [0]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
   
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
    # Fl correction?
    scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
    manual_scale_factor = manual_scale_factors_v4[sample_name]
    trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
    para_DA = para_DA - trmsn90_corr
    perp_DA = perp_DA - trmsn90_corr

    # slice ISI data
    para_ISI = para_DA.interpolate_na(dim='q').sel(q=slice(qslice[0], qslice[1])).mean('chi').mean('q')
    perp_ISI = perp_DA.interpolate_na(dim='q').sel(q=slice(qslice[0], qslice[1])).mean('chi').mean('q')

    # plot
    fig, ax = plt.subplots(figsize=figsize, dpi=120)

    para_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='$\parallel$', yscale='log')
    perp_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='$\perp$', yscale='log')
    fig.suptitle(f'ISI (FL Corrected): {sample_name}', fontsize=14, x=0.55)
    ax.set(title=f'Pol = {pol}°, chi width = {chi_width}°, Q = ({qslice[0]}, {qslice[1]}) ' + 'Å$^{-1}$', 
           xlabel='X-ray energy [eV]', ylabel='Intensity [arb. units]')
    ax.legend(loc='upper right', fontsize=14)
    ax.grid(axis='x')
    plt.subplots_adjust(top=0.86, bottom=0.2, left=0.2)
    
    plt.show()
    # fig.savefig(outPath.joinpath('prsoxs_plots', 'carbon', 'isi_carbon_v4', 
    #     f'{sample_name}_{edge}_{intensity_type}_chi{chi_width}deg_q{qslice[0]}-{qslice[1]}_pol{pol}deg.png'), dpi=120)

    plt.close('all')

In [ ]:
selected_samples = ['Y6_3000_dSiN', 'PM6_3000_dSiN', 'PM6-Y6_3000_dSiN', 'PM7-Y6_3000_dSiN', 'Y7_3000_dSiN', 'PM7_3000_dSiN', 'PM7-Y7_3000_dSiN']
# selected_samples = ['Y6_3000_dSiN', 'PM6_3000_dSiN', 'PM6-Y6_3000_dSiN', 'PM7-Y6_3000_dSiN', 'Y7_3000_dSiN', 'PM7_3000_dSiN', 'PM7-Y7_3000_dSiN']

### 3. 2D Linecut Maps

In [ ]:
# make selection
edge = 'carbon'
intensity_type = 'corr'
# intensity_type = 'corr-sin-fl'
qlims = (9e-3, None)
chi_width = 90

# energy_min = 260
# energy_max = 350

energy_min = 282
energy_max = 292

for sample_name in tqdm(filtered_selected_samples):
# for sample_name in tqdm(rsoxs_datasets['polar_carbon'].sample_name.values[4:]):
    # for pol in [0]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
   
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
    # Fl correction?
    scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
    manual_scale_factor = manual_scale_factors_v4[sample_name]
    trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
    para_DA = para_DA - trmsn90_corr
    perp_DA = perp_DA - trmsn90_corr
                
    # Plot
    fig, axs = plt.subplots(1, 2, figsize=(11,5))
    cmin = 3e8
    cmax = 1e11

    para_slice = para_DA.mean('chi').sel(q=slice(None,0.105), energy=slice(energy_min,energy_max))  # .plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice = perp_DA.mean('chi').sel(q=slice(None,0.105), energy=slice(energy_min,energy_max))  # .plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # cmin = para_slice.compute().quantile(0.01)
    # cmax = para_slice.compute().quantile(0.995)

    para_slice.plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
    perp_slice.plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

    # Add colorbar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
    cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    cbar.set_label(label='Double-Norm-Corrected Intensity [arb. units]', labelpad=12, rotation=270)

    fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
    # fig.suptitle(f'Linecut Maps: {sample_name} Bare SiN & Fluorescence Subtracted, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)

    fig.set(tight_layout=True)

    axs[0].set(xscale='log', title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
    axs[1].set(xscale='log', title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')

    fig.savefig(plotsPath.joinpath('linecut_maps_carbon_v2', f'{sample_name}_{edge}_{energy_min}-{energy_max}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)

    # plt.show()
    plt.close('all')

In [ ]:
# a = para_EvsQ_avg.hvplot.line(groupby="energy", loglog=True)
# a * perp_EvsQ_avg.hvplot.line(groupby="energy")

### 4. Meaned 1D intensity lineplots, selected energies

In [ ]:
full_DS = rsoxs_datasets['polar_carbon']
full_DS

In [ ]:
# Previous code to subtract SiN:

#     sin_para_DA, sin_perp_DA = make_para_perp_DAs(rsoxs_datasets, 'BareSiN_1mm', edge, intensity_type, pol, qlims, chi_width)  
#     para_sin_min_q_DA = sin_para_DA.mean('chi').sel(q=slice(0.02, 0.08)).min('q')
#     perp_sin_min_q_DA = sin_perp_DA.mean('chi').sel(q=slice(0.02, 0.08)).min('q')

#     para_min_q_DA = para_DA.mean('chi').sel(q=slice(0.02, 0.08)).min('q')
#     perp_min_q_DA = perp_DA.mean('chi').sel(q=slice(0.02, 0.08)).min('q')

#     para_sin_subtract_weightings = (para_sin_min_q_DA/para_min_q_DA)*1
#     perp_sin_subtract_weightings = (perp_sin_min_q_DA/perp_min_q_DA)*1

#     para_DA = para_DA - (para_sin_subtract_weightings * sin_para_DA)
#     perp_DA = perp_DA - (perp_sin_subtract_weightings * sin_perp_DA)   

In [ ]:
trmsn90_bkgs_DA = xr.concat(trmsn90_bkgs, dim='sample_name')
trmsn90_bkgs_DA

In [ ]:
for sample_name in trmsn90_bkgs_DA.sample_name.values:
    trmsn90_bkgs_DA.sel(sample_name=sample_name).plot()
    plt.show()

In [ ]:
fl_bkgs_DA = xr.concat(fl_bkgs, dim='sample_name')
fl_bkgs_DA

In [ ]:
trmsn90_corr.plot()

In [ ]:
manual_scale_factors_v1 = {
    'PM6_CB_3000': 6.5,
    'PM6_CBCN': 8,
    'PM6_CF_3000': 6,
    'PM6_CFCN': 9,
    'PM6-Y6_CB': 6.2,
    'PM6-Y6_CBCN': 5.5,
    'PM6-Y6_CF': 6,
    'PM6-Y6_CFCN': 8,
    'PM6-Y6BO_CB': 6.4,
    'PM6-Y6BO_CF': 5.8,
    'PM6-Y6BO_CFCN': 8,
    'PM6-Y7_CB': 6.1,
    'PM6-Y7_CBCN': 8,
    'PM6-Y7_CF': 5.9,
    'PM6-Y7_CFCN': 8,
    'PM6-Y7BO_CB': 5.5,
    'PM6-Y7BO_CBCN': 6.3,
    'PM6-Y7BO_CF': 5.8,
    'PM6-Y7BO_CFCN': 7,
    'Y6_CB_2000_1': 5.5,
    'Y6_CB_2000': 6,
    'Y6_CB_2500': 6,
    'Y6_CB_3000': 6,
    'Y6_CF_2000': 8,
    'Y6_CF_2500': 8.5,
}

len(manual_scale_factors_v1)

In [ ]:
manual_scale_factors_v2 = {
    'PM6_CB_3000': 9,
    'PM6_CBCN': 12,
    'PM6_CF_3000': 6,
    'PM6_CFCN': 12,
    'PM6-Y6_CB': 6.5,
    'PM6-Y6_CBCN': 4,
    'PM6-Y6_CF': 6,
    'PM6-Y6_CFCN': 12,
    'PM6-Y6BO_CB': 8,
    'PM6-Y6BO_CF': 5,
    'PM6-Y6BO_CFCN': 12,
    'PM6-Y7_CB': 6.5,
    'PM6-Y7_CBCN': 8.5,
    'PM6-Y7_CF': 5,
    'PM6-Y7_CFCN': 14,
    'PM6-Y7BO_CB': 5,
    'PM6-Y7BO_CBCN': 8,
    'PM6-Y7BO_CF': 5,
    'PM6-Y7BO_CFCN': 12,
    'Y6_CB_2000_1': 5,
    'Y6_CB_2000': 6,
    'Y6_CB_2500': 6,
    'Y6_CB_3000': 6,
    'Y6_CF_2000': 15,
    'Y6_CF_2500': 17,
}

len(manual_scale_factors_v2)

In [ ]:
manual_scale_factors_v3 = {
    'PM6_CB_3000': 8,
    'PM6_CBCN': 10.5,
    'PM6_CF_3000': 6,
    'PM6_CFCN': 12,
    'PM6-Y6_CB': 6.3,
    'PM6-Y6_CBCN': 5,
    'PM6-Y6_CF': 6,
    'PM6-Y6_CFCN': 11,
    'PM6-Y6BO_CB': 7,
    'PM6-Y6BO_CF': 5,
    'PM6-Y6BO_CFCN': 12.5,
    'PM6-Y7_CB': 6.5,
    'PM6-Y7_CBCN': 8.5,
    'PM6-Y7_CF': 4.5,
    'PM6-Y7_CFCN': 12,
    'PM6-Y7BO_CB': 5,
    'PM6-Y7BO_CBCN': 8.2,
    'PM6-Y7BO_CF': 5,
    'PM6-Y7BO_CFCN': 10,
    'Y6_CB_2000_1': 5,
    'Y6_CB_2000': 6,
    'Y6_CB_2500': 6,
    'Y6_CB_3000': 6,
    'Y6_CF_2000': 18,
    'Y6_CF_2500': 20,
}

len(manual_scale_factors_v3)

In [ ]:
manual_scale_factors_v4 = {
    'PM6_CB_3000': 8.4,
    'PM6_CBCN': 11,
    'PM6_CF_3000': 6,
    'PM6_CFCN': 10.5,
    'PM6-Y6_CB': 6.3,
    'PM6-Y6_CBCN': 4.5,
    'PM6-Y6_CF': 6,
    'PM6-Y6_CFCN': 11.5,
    'PM6-Y6BO_CB': 7.5,
    'PM6-Y6BO_CF': 5,
    'PM6-Y6BO_CFCN': 12.8,
    'PM6-Y7_CB': 6.5,
    'PM6-Y7_CBCN': 8.5,
    'PM6-Y7_CF': 4.5,
    'PM6-Y7_CFCN': 12.8,
    'PM6-Y7BO_CB': 5,
    'PM6-Y7BO_CBCN': 8.2,
    'PM6-Y7BO_CF': 5,
    'PM6-Y7BO_CFCN': 10,
    'Y6_CB_2000_1': 5,
    'Y6_CB_2000': 6,
    'Y6_CB_2500': 6,
    'Y6_CB_3000': 6,
    'Y6_CF_2000': 18,
    'Y6_CF_2500': 20,
}

len(manual_scale_factors_v4)

In [ ]:
filtered_selected_samples = selected_samples[:19] + selected_samples[22:26] + selected_samples[27:29]
len(filtered_selected_samples)

In [ ]:
filtered_selected_samples = ['PM6_CB_3000', 'PM6_CBCN', 'PM6_CF_3000', 'PM6_CFCN']

In [ ]:
# make selection
edge = 'carbon'
intensity_type = 'corr'
qlims = (8e-3, None)
# qlims = (None,None)
chi_width = 90

for sample_name in tqdm(filtered_selected_samples):
# for sample_name in tqdm(rsoxs_datasets['polar_carbon'].sample_name.values[:]):
# for sample_name in tqdm(['PM6-Y6_CB', 'PM6-Y6BO_CB', 'PM6-Y6BO_CF']):
    # for pol in [0]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
   
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 
    
    # Fl correction?
    scale_factor = float(para_DA.sel(energy=slice(285,305), q=slice(2e-2,6e-2)).mean('chi').mean('energy').integrate('q'))
    manual_scale_factor = manual_scale_factors_v4[sample_name]
    trmsn90_corr = (manual_scale_factor * scale_factor * trmsn90_bkgs_DA.sel(sample_name=sample_name))
    
    para_DA = para_DA - trmsn90_corr
    perp_DA = perp_DA - trmsn90_corr
        
    # Plot
    energies = para_DA.energy.sel(energy=[275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 288, 300, 305], method='nearest').data

    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))

    fig, axs = plt.subplots(ncols=2,figsize=(8,5), tight_layout=True)

    qmin = qlims[0]
    qmax = qlims[1]

    for j, energy in enumerate(energies):
        (para_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
         .plot.line(ax=axs[0], color=colors[j], yscale='log', xscale='log', label=energy))
        (perp_DA.sel(q=slice(qmin, qmax), energy=energy).mean('chi')
         .plot.line(ax=axs[1], color=colors[j], yscale='log', xscale='log', label=energy))

    # fig.suptitle(f'IvsQ, {pol}° pol, 90° chi width: {sample_name}', x=0.47)
    fig.suptitle(f'IvsQ ($Fl$ Corr (C={manual_scale_factor})), {pol}° pol, 90° chi width: {sample_name}', x=0.47)

    axs[0].set(title=f'Parallel to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[0].set_ylim(bottom=1e8)
    axs[1].set(title=f'Perpendicular to E$_p$', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    axs[1].set_ylim(bottom=1e8)
    axs[1].legend(title='Energy [eV]', loc=(1.05,0.1))

    # savePath = plotsPath.joinpath('fl_carbon_backgrounds', f'manual_scale_factor_dict-v4')
    # savePath.mkdir(parents=True, exist_ok=True)
    # fig.savefig(savePath.joinpath(f'{sample_name}_chiWidth-{chi_width}deg_q{qmin}-{qmax}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')

### Para - perp log scale intensity

In [ ]:
%matplotlib inline

In [ ]:
plotsPath

In [ ]:
# make selection
edge = 'carbon'
DS = rsoxs_datasets[f'polar_{edge}']
# DS = rsoxs_datasets[f'polar_{edge}_tilted']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.01, None)
e_slice = slice(282, 292)

selected_samples = ['PM6_CBCN', 'PM6_CB_3000', 'PM6_CFCN', 'PM6_CF_3000',] 

for sample_name in tqdm(DS.sample_name.values[:]):
# for sample_name in tqdm(selected_samples):
    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    # ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
    ar_DA = np.abs(para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   
    energies = para_DA.energy.sel(energy=[260, 283, 284.4, 284.8, 285.2, 286.2, 338.75], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    for j, energy in enumerate(energies):
        (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log', yscale='log',
                                            linestyle='None', marker='.', markersize=3))

    fig.suptitle(f'$\parallel$ - $\perp$ Intensity Absolute Difference: {sample_name}', x=0.46, y=0.95)

    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(1e8, None), ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(None, None), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    legend = ax.legend(title='Energy [eV]', loc=(1.03,0.23))
    for handle in legend.legend_handles:
        handle.set_markersize(15)
    
    savePath = plotsPath.joinpath('carbon/para-perp_cuts_v1')
    savePath.mkdir(exist_ok=True)
    fig.savefig(savePath.joinpath( 
        f'{sample_name}_{intensity_type}_chiWidth-{chi_width}deg_q-{q_slice.start}-{q_slice.stop}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')

### Load solution SAXS PM6 data text files

In [ ]:
# Check files
saxsPath = pathlib.Path('/nsls2/data/sst/proposals/2023-3/pass-313412/processed_data/prsoxs_plots/carbon/PM6_solution_saxs_txts')
# saxsPath = outPath.joinpath('prsoxs_plots/carbon/PM6_solution_saxs_txts')
display([f.name for f in sorted(saxsPath.glob('*'))[:3]])
display([f.name for f in sorted(saxsPath.glob('*'))[3:]])

In [ ]:
# plt.close('all')
# sigmas = [1, 2, 3, 4]
# colors = plt.cm.viridis(np.linspace(0,0.85,len(sigmas)))
# for arr in [pm6_cb, pm6_cf, pm6_cn]:
#     fig, ax = plt.subplots()
#     for i, sigma in enumerate(sigmas[::1]):
#         ax.plot(gaussian_filter1d(arr[:,1], sigma), color=colors[i], label=sigma)
#         ax.set(xscale='log', yscale='log')
#         ax.legend(title='sigma')

#     # plt.show()
#     # plt.close('all')

In [ ]:
from scipy.ndimage import gaussian_filter1d

In [ ]:
# Load data into numpy arrays
pm6_cb, pm6_cf, pm6_cn = [np.loadtxt(f) for f in sorted(saxsPath.glob('*'))[:3]]
y6_cn, y6_to, y7_cb, y7_cn = [np.loadtxt(f) for f in sorted(saxsPath.glob('*'))[3:]]

# Smooth saxs with gaussian filter:
sigma = 3  # In pixels
pm6_cb[:,1], pm6_cf[:,1], pm6_cn[:,1] = [gaussian_filter1d(arr[:,1], sigma) for arr in [pm6_cb, pm6_cf, pm6_cn]]
y6_cn[:,1], y6_to[:,1], y7_cb[:,1], y7_cn[:,1] = [gaussian_filter1d(arr[:,1], sigma) for arr in [y6_cn, y6_to, y7_cb, y7_cn]]

# Below should really just be a loop... made sense for 3 samples
# Put numpy arrays into DataArrays
pm6_cb_DA = xr.DataArray(data=pm6_cb[:,1], dims=['q'], coords={'q':pm6_cb[:,0], 'err':('q', pm6_cb[:,2])})
pm6_cf_DA = xr.DataArray(data=pm6_cf[:,1], dims=['q'], coords={'q':pm6_cf[:,0], 'err':('q', pm6_cf[:,2])})
pm6_cn_DA = xr.DataArray(data=pm6_cn[:,1], dims=['q'], coords={'q':pm6_cn[:,0], 'err':('q', pm6_cn[:,2])})                                              
y6_cn_DA = xr.DataArray(data=y6_cn[:,1], dims=['q'], coords={'q':y6_cn[:,0], 'err':('q', y6_cn[:,2])})
y6_to_DA = xr.DataArray(data=y6_to[:,1], dims=['q'], coords={'q':y6_to[:,0], 'err':('q', y6_to[:,2])})
y7_cb_DA = xr.DataArray(data=y7_cb[:,1], dims=['q'], coords={'q':y7_cb[:,0], 'err':('q', y7_cb[:,2])})
y7_cn_DA = xr.DataArray(data=y7_cn[:,1], dims=['q'], coords={'q':y7_cn[:,0], 'err':('q', y7_cn[:,2])})
                                                  
# Move DataArrays into Dataset
saxs_DS = xr.Dataset(attrs={'name':'PM6_Solution_SAXS'})
saxs_DS['PM6_CB'] = pm6_cb_DA
saxs_DS['PM6_CF'] = pm6_cf_DA
saxs_DS['PM6_CN'] = pm6_cn_DA                                            
saxs_DS['Y6_CN'] = y6_cn_DA
saxs_DS['Y6_TO'] = y6_to_DA
saxs_DS['Y7_CB'] = y7_cb_DA
saxs_DS['Y7_CN'] = y7_cn_DA

saxs_DS

In [ ]:
# # Ensure q values are equal between datasets, else interpolation needed
# print(np.array_equal(pm6_cb[:,0], pm6_cf[:,0]))
# print(np.array_equal(pm6_cf[:,0], pm6_cn[:,0]))

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
for sample in ['PM6_CB', 'PM6_CF', 'PM6_CN']:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

In [ ]:
%matplotlib widget

In [ ]:
# qmin = 0.008
qmin = None
# qmax = float(full_DS.q.max())
qmax = None

plt.close('all')
fig, ax = plt.subplots()
for sample in ['Y6_CN', 'Y7_CB', 'Y7_CN']:
# for sample in ['Y7_CB', 'Y7_CN']:
    saxs_DS[sample].sel(q=slice(qmin,qmax)).plot.line(ax=ax, xscale='log',yscale='log', label=sample)

ax.set_ybound(lower=1e-5)
ax.legend()
plt.show()

### Plot para-perp P-RSoXS + solution SAXS overlays

In [ ]:
%matplotlib inline

In [ ]:
%matplotlib widget

In [ ]:
# make selection
edge = 'carbon'
DS = rsoxs_datasets[f'polar_{edge}']

intensity_type = 'corr'
chi_width = 90
q_slice = slice(0.01, None)
# e_slice = slice(282, 292)
energy = 285.2
# energy = 284.8
energies = [260, 283, 284.4, 284.8, 285.2, 286.2, 338.75]
shift_qval = 0.015  # value by which to look to shift data, divide rsoxs by this value
# shift_qval = 0.02  # value by which to look to shift data, divide rsoxs by this value
# shift_qval = 0.017  # value by which to look to shift data, divide rsoxs by this value

# selected_samples = ['PM6_CBCN', 'PM6_CB_3000', 'PM6_CFCN', 'PM6_CF_3000'] 
# selected_samples = ['PM6_CF_3000','PM6_CFCN'] 
selected_samples = ['PM6_CB_3000','PM6_CBCN'] 

saxs_samples = ['PM6_CF', 'PM6_CB', 'PM6_CN']
# saxs_samples = ['Y6_CN', 'Y7_CN', 'Y7_CB']
saxs_sample_to_align = 'PM6_CN'
selected_qval = float(saxs_DS[saxs_sample_to_align].sel(q=shift_qval, method='nearest'))

# Shift all saxs to line up with selected sample qval
factor1 = selected_qval / float(saxs_DS['PM6_CB'].sel(q=shift_qval, method='nearest'))
factor2 = selected_qval / float(saxs_DS['PM6_CF'].sel(q=shift_qval, method='nearest'))
pm6_cb_shifted = saxs_DS['PM6_CB'] * factor1
pm6_cf_shifted = saxs_DS['PM6_CF'] * factor2

# factor1 = selected_qval / float(saxs_DS['Y7_CN'].sel(q=shift_qval, method='nearest'))
# factor2 = selected_qval / float(saxs_DS['Y7_CB'].sel(q=shift_qval, method='nearest'))
# y7_cn_shifted = saxs_DS['Y7_CN'] * factor1
# y7_cb_shifted = saxs_DS['Y7_CB'] * factor2

saxs_colors = plt.cm.RdPu(np.linspace(0.4,0.9,len(saxs_samples)))
plt.close('all')

# for selected_samples in (['Y6_CF', 'Y6_p5CN-CF'],
#                          ['Y6_4CF-1CB', 'Y6_2CF-3CB', 'Y6_p5CN-2CF-3CB'], 
#                          ['Y6_CB', 'Y6_p5CN-CB']):
# for selected_samples in (['PM6_CB', 'PM6_p5CN-CB', 'PM6_1CN-CB', 'PM6_5CN-CB'],
#                          ['PM6_CF', 'PM6_p5CN-CF', 'PM6_1CN-CF', 'PM6_5CN-CF'],
#                          ['PM6_4CF-1CB', 'PM6_2CF-3CB', 'PM6_p5CN-2CF-3CB']):
rsoxs_colors = plt.cm.GnBu(np.linspace(0.4,1,len(selected_samples)))

for energy in tqdm(energies, desc='Energies'):
    fig, ax = plt.subplots(tight_layout=True, figsize=(6,6), dpi=130)
    for i, sample_name in enumerate(tqdm(selected_samples)):
        pol_paras = []
        pol_perps = []
        for pol in [0, 90]:
            para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, (q_slice.start,q_slice.stop), chi_width) 
            pol_paras.append(para_DA)
            pol_perps.append(perp_DA)


        pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
        pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
        pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
        pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

        pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
        pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

        para_DA = (pol_paras[0] + pol_paras[1])/2
        perp_DA = (pol_perps[0] + pol_perps[1])/2

        para_DA = para_DA.assign_coords({'polarization':'avg'})
        perp_DA = perp_DA.assign_coords({'polarization':'avg'})

        para_DA = para_DA.interpolate_na(dim='q')
        perp_DA = perp_DA.interpolate_na(dim='q')

        pol = str(para_DA.polarization.values) 

        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi'))  # / (para_DA.mean('chi') + perp_DA.mean('chi'))
        DA = ar_DA.sel(energy=energy, method='nearest')

        # Shift RSoXS to line up with SAXS
        rsoxs_qval = float(DA.sel(q=shift_qval, method='nearest'))
        rsoxs_shift_factor = selected_qval / rsoxs_qval
        DA = DA * rsoxs_shift_factor

        # Plot rsoxs diff profile:
        DA.plot.line(ax=ax, color=rsoxs_colors[i], label=f'{energy} eV: $\parallel - \perp$ {sample_name}', 
                     linestyle='None', marker='.', markersize=2)

    # pm6_cf_shifted.plot.line(ax=ax, color=saxs_colors[0], label='Solution: PM6 in CF', zorder=10)    
    pm6_cb_shifted.plot.line(ax=ax, color=saxs_colors[1], label='Solution: PM6 in CB')
    # y7_cn_shifted.plot.line(ax=ax, color=saxs_colors[1], label='Solution: Y7 in CN') 
    # y7_cb_shifted.plot.line(ax=ax, color=saxs_colors[0], label='Solution: Y7 in CB') 
    saxs_DS[saxs_sample_to_align].plot.line(ax=ax, color=saxs_colors[2], label='Solution: PM6 in CN') 

    ax.set(xscale='log', yscale='log', ylabel='Intensity [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.set(title=f'$\parallel - \perp$ RSoXS overlayed on solution SAXS; {chi_width}° $\\chi$ wedges', ylim=(None, None), xlim=(None, 0.4))
    # ax.legend(title='Sample', loc=(1.02,0.2))
    AA_str = '$Å^{-1}$'
    legend = ax.legend(title=f'Data Plotted (aligned at Q={shift_qval} {AA_str})')
    for handle in legend.legend_handles:
        handle.set_markersize(15)

    plt.show()
    # plt.close('all')

## Anisotropy Ratio Plots

### 1. 2D AR Maps

In [ ]:
rsoxs_datasets['']

In [ ]:
# make selection
# sample_name = 'Y6_CB_2500'
edge = 'carbon'
intensity_type = 'corr'
# pol = 90
qlims = (9e-3, None)
chi_width = 90
energy_min = 282
energy_max = 330

for sample_name in tqdm(sorted(selected_samples)):
# for sample_name in tqdm(filtered_selected_samples):
# for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.values[:]):
    # for pol in [0,90]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)   

    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot
    vlim = 0.5
    ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot.pcolormesh(figsize=(8,5), norm=plt.Normalize(-vlim, vlim))
    # ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5))

    ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
    ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]', xscale='log')
    ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)


    # ax.figure.savefig(plotsPath.joinpath('ar_maps_carbon_v2', 
    #     f'vset__{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_q-{qlims[0]}-{qlims[1]}_energy{energy_min}-{energy_max}_pol{pol}deg.png'), 
    #                   dpi=120)

    plt.show()
    plt.close('all')

### 2. 1D meaned AR linecuts

In [ ]:
# make selection
# sample_name = 'Y6_CB_2500'
edge = 'carbon'
intensity_type = 'corr'
# pol = 90
qlims = (8e-3, 0.105)
chi_width = 90
energy_min = 282
energy_max = 292

for sample_name in tqdm(filtered_selected_samples):
# for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.values[:]):
    # for pol in [0,90]:
    #     para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)   

    pol_paras = []
    pol_perps = []
    for pol in [0, 90]:
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width) 
        pol_paras.append(para_DA)
        pol_perps.append(perp_DA)


    pol_paras[0] = pol_paras[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_paras[1] = pol_paras[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_paras[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[0].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})
    pol_perps[1] = pol_perps[1].assign_coords({'chi': np.linspace(0, chi_width, len(pol_perps[1].chi.values))})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    pol_paras[0] = pol_paras[0].interp({'chi': pol_paras[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_paras[1].chi.values))})
    pol_perps[0] = pol_perps[0].interp({'chi': pol_perps[1].chi.values})  # .assign_coords({'chi': np.linspace(0, 90,len(pol_perps[1].chi.values))})

    para_DA = (pol_paras[0] + pol_paras[1])/2
    perp_DA = (pol_perps[0] + pol_perps[1])/2

    para_DA = para_DA.assign_coords({'polarization':'avg'})
    perp_DA = perp_DA.assign_coords({'polarization':'avg'})

    para_DA = para_DA.interpolate_na(dim='q')
    perp_DA = perp_DA.interpolate_na(dim='q')

    pol = str(para_DA.polarization.values) 

    # Select AR data
    ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) # / (para_DA.mean('chi') + perp_DA.mean('chi'))

    # Plot   

    # energies = para_DA.energy.sel(energy=[390, 398.8, 399.8, 401.1, 410], method='nearest').data
    energies = para_DA.energy.sel(energy=[275, 283, 284, 284.4, 284.8, 285.2, 285.6, 286.2, 287, 288, 300, 305], method='nearest').data
    # energies = para_DA.energy.sel(energy=[283, 284.2, 284.4, 284.8, 285.1, 288, 300, 305, 350], method='nearest').data
    # energies = para_DA.energy.sel(energy=[283, 284.2, 284.8, 285.1, 286.2, 287, 288, 300], method='nearest').data
    # energies = para_DA.energy.sel(energy=[283, 284.2, 284.4, 284.8, 285.1, 285.8, 286.2, 287, 288, 300], method='nearest').data
    cmap = plt.cm.turbo.copy()
    colors = cmap(np.linspace(0, 1, len(energies)))        

    # # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=energies[0], vmax=energies[-1]))
    # cax = ax.inset_axes([1.03, 0, 0.03, 1])
    # cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
    # cbar.set_label(label=f'Time [seconds]', labelpad=14)
    # cbar.set_ticks(np.round(np.linspace(energies[0], energies[-1], len(energies)), 2))

    fig, ax = plt.subplots(tight_layout=True, figsize=(6,4), dpi=120)

    qmin = 8e-3 
    qmax = 8e-2

    for j, energy in enumerate(energies):
        (ar_DA.sel(energy=energy).plot.line(ax=ax, color=colors[j], label=energy, xscale='log', yscale='log'))

    # fig.suptitle(f'Anisotropy Ratio Linecuts: {sample_name}', x=0.46)
    fig.suptitle(f'Parallel minus perpendicular P-RSoXS: {sample_name}', x=0.46)

    # ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(-0.5, 0.5), ylabel='AR [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.set(title=f'Chi width = {chi_width}°, Pol = {pol}°', ylim=(1e8, None), ylabel='I [arb. units]', xlabel='Q [$Å^{-1}$]')
    ax.legend(title='Energy [eV]', loc=(1.03,0.02))


    # fig.savefig(plotsPath.joinpath('selected_ar_linecuts_carbon_v2', 
    #     f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_q-{qmin}-{qmax}_pol{pol}deg.png'), dpi=120)

    plt.show()
    plt.close('all')

In [ ]:
ar_DA

In [ ]:
ar_DA.hvplot.line(groupby="energy")

## Loop for saving many plots

In [ ]:
rsoxs_datasets['polar_oxygen']

In [ ]:
# make selection
edge = 'oxygen'
intensity_type = 'corr'
qlims = (0.01, 0.08)
chi_width = 30
energy_min = 525
energy_max = 545
cmin = 5e8
cmax = 5e10

for sample_name in tqdm(rsoxs_datasets[f'polar_{edge}'].sample_name.data):
    for pol in [0, 90]:
        ### Select para & perp DataArrays
        para_DA, perp_DA = make_para_perp_DAs(rsoxs_datasets, sample_name, edge, intensity_type, pol, qlims, chi_width)  
        
        ### ISI:
        # Slice ISI data
        para_ISI = para_DA.interpolate_na(dim='q').mean('chi').sum('q')
        perp_ISI = perp_DA.interpolate_na(dim='q').mean('chi').sum('q')
        
        # Plot
        fig, ax = plt.subplots()
        para_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='para', yscale='log')
        perp_ISI.sel(energy=slice(energy_min,energy_max)).plot.line(ax=ax, label='perp', yscale='log')
        # para_ISI.plot.line(ax=ax, label='para', yscale='log')
        # perp_ISI.plot.line(ax=ax, label='perp', yscale='log')        
        fig.suptitle('Integrated Scattering Intensity (ISI)', fontsize=14)
        ax.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', xlabel='Photon Energy [eV]', ylabel='Double-Norm-Corrected Intensity [arb. units]')
        ax.legend()
        fig.savefig(plotsPath.joinpath('isi_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')
        
        ### Linecut Maps:
        fig, axs = plt.subplots(1, 2, figsize=(11,5))

        para_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[0], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)
        perp_DA.mean('chi').sel(energy=slice(energy_min,energy_max)).plot(ax=axs[1], cmap=cmap, norm=LogNorm(cmin, cmax), add_colorbar=False)

        sm = plt.cm.ScalarMappable(cmap=cmap, norm=LogNorm(cmin, cmax)) # Create a ScalarMappable object with the colormap and normalization & add the colorbar to the figure
        cax = axs[1].inset_axes([1.03, 0, 0.05, 1])
        cbar = fig.colorbar(sm, cax=cax, orientation='vertical')
        cbar.set_label(label='Intensity [arb. units]', labelpad=12)
        fig.suptitle(f'Linecut Maps: {sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', fontsize=14)
        fig.set(tight_layout=True)
        axs[0].set(title='Parallel to $E_p$', ylabel='Photon energy [eV]', xlabel='q [$Å^{-1}$]')
        axs[1].set(title='Perpendicular to $E_p$ ', ylabel=None, xlabel='q [$Å^{-1}$]')
        fig.savefig(plotsPath.joinpath('linecut_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

        ### AR Maps:
        # Select AR data
        ar_DA = (para_DA.mean('chi') - perp_DA.mean('chi')) / (para_DA.mean('chi') + perp_DA.mean('chi'))

        if para_DA.sample_name=='BareSiN':
            bareSiN_ar_DA = ar_DA.copy()
            
        # # Subtract AR from SiN:
        # ar_DA = ar_DA - bareSiN_ar_DA

        # Plot
        ax = ar_DA.sel(energy=slice(energy_min,energy_max)).plot(figsize=(8,5), norm=plt.Normalize(-0.6, 0.6))
        ax.figure.suptitle('Anisotropy Ratio (AR) Map', fontsize=14, x=0.43)
        ax.axes.set(title=f'{sample_name}, Polarization = {pol}°, Chi Width = {chi_width}°', ylabel='Photon Energy [eV]', xlabel='q [$Å^{-1}$]')
        ax.colorbar.set_label('AR [arb. units]', rotation=270, labelpad=12)
        ax.figure.savefig(plotsPath.joinpath('ar_maps_oxygen_v1', f'{sample_name}_{edge}_{intensity_type}_chiWidth-{chi_width}deg_pol{pol}deg.png'), dpi=120)
        # plt.show()
        plt.close('all')

## Incorporating simulation output
old and incomplete

In [ ]:
import h5py

In [ ]:
whos PosixPath

In [ ]:
simsPath = rootPath.joinpath('imgs_analysis/sim_runs')

In [ ]:
[f.name for f in simPath.iterdir()]

In [ ]:
simPath = simsPath.joinpath('D1.0_a1.0_eps1.0_250pix_375size_0.5m_600steps_1')
h5path = simPath.joinpath('HDF5')
h5list = sorted(h5path.glob('E*h5'))

In [ ]:
def print_key(f, key):
    try:
        keys2 = f[key].keys()
        for key2 in keys2:
            new_key = key + '/' + key2
            print_key(f, new_key)
    except AttributeError:
        print(key)

with h5py.File(h5list[0],'r') as f:
    for key in f.keys():
        print_key(f, key)

#### The scattering pattern is contained within 'K0/projection.' If you simulated multiple incident angles there would be additional 'K#' keys. KIDList contains the vector values for each of the incident angles (propagation direction).

#### Instantiate the PyHyperScattering loader and integrator

In [ ]:
load = phs.load.cyrsoxsLoader(use_chunked_loading=True)
integ = phs.integrate.WPIntegrator(force_np_backend=True) # avoiding gpu backend for this tutorial

In [ ]:
raw = load.loadDirectory(simPath)

In [ ]:
raw

In [ ]:
remeshed = integ.integrateImageStack(raw)

In [ ]:
remeshed

In [ ]:
remeshed.sel(energy=285.1).plot()

In [ ]:
# c = cm.jet(np.linspace(0,1,len(remeshed)))

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(10,3),dpi=140,constrained_layout=True)
raw.sel(energy=275).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[0],add_colorbar=False)
raw.sel(energy=285).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[1],add_colorbar=False)
raw.sel(energy=295).plot(norm=LogNorm(1e-7,1),cmap='terrain',ax=ax[2])

[{axes.set_xlim(-0.1,0.1),axes.set_ylim(-0.1,0.1)} for axes in ax]
plt.show()

In [ ]:
loaded_rsoxs.build_tiff_stack??